In [1]:
from brian2cuda import *
from brian2 import *
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
set_device("cuda_standalone")

In [2]:
def get_connectivity(n_pre, n_post, radius):
    P_to_E = np.zeros( (n_pre, n_post) )
    for e_num in range(n_post):
        N_pre_2D = int(np.sqrt(n_pre))

        #choose a center
        x_center = np.random.randint(0,N_pre_2D)
        y_center = np.random.randint(0,N_pre_2D)

        #fill a circle around that center
        for neur_num in range(n_pre):
            #compute neurons 2D's coordinates
            x_neur, y_neur = get_2D_coords(neur_num, n_pre)
            if ((x_neur-x_center)**2 + (y_neur-y_center)**2) < radius**2:
                P_to_E[neur_num, e_num] = 1
    return(P_to_E)

def get_2D_coords(ind, N):
    """
    1D to 2D coordinate on a square
    """
    N_pre_2D = int(np.sqrt(N))
    return(ind // N_pre_2D , ind % N_pre_2D)

In [3]:
def load_plasticity_rule(rule):
    
    # 6 params per each connection
    tau_preSTDP_EE = rule[0]
    tau_postSTDP_EE = rule[1]
    alpha_EE = rule[2]
    beta_EE = rule[3]
    gamma_EE = rule[4]
    kappa_EE = rule[5]

    tau_preSTDP_EI = rule[6]
    tau_postSTDP_EI = rule[7]
    alpha_EI = rule[8]
    beta_EI = rule[9]
    gamma_EI = rule[10]
    kappa_EI = rule[11]

    tau_preSTDP_IE = rule[12]
    tau_postSTDP_IE = rule[13]
    alpha_IE = rule[14]
    beta_IE = rule[15]
    gamma_IE = rule[16]
    kappa_IE = rule[17]

    tau_preSTDP_II = rule[18]
    tau_postSTDP_II = rule[19]
    alpha_II = rule[20]
    beta_II = rule[21]
    gamma_II = rule[22]
    kappa_II = rule[23]

    return tau_preSTDP_EE, tau_postSTDP_EE, alpha_EE, beta_EE, gamma_EE, kappa_EE, tau_preSTDP_EI, tau_postSTDP_EI, alpha_EI, beta_EI, gamma_EI, kappa_EI, tau_preSTDP_IE, tau_postSTDP_IE, alpha_IE, beta_IE, gamma_IE, kappa_IE, tau_preSTDP_II, tau_postSTDP_II, alpha_II, beta_II, gamma_II, kappa_II 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_surrounding_indices(x, y, spread=4, grid_size=64):
    indices = []
    points = []

    x_start, x_end = max(0, x - spread), min(grid_size - 1, x + spread)
    y_start, y_end = max(0, y - spread), min(grid_size - 1, y + spread)

    for i in range(x_start, x_end + 1):
        for j in range(y_start, y_end + 1):
            points.append((i, j))
            indices.append(i*64 + j)

    return points, indices

grid_size = 64
center_x, center_y = 10, 10

points, indices = get_surrounding_indices(center_x, center_y)

x_vals, y_vals = zip(*points)

In [ ]:
start_scope()

PtE = get_connectivity(10000, 4096, 8)
PtI = get_connectivity(10000, 1024, 8)

NE = 4096
NI = 1024
tau_ampa = 5*ms
tau_gaba = 10*ms
tau_nmda = 100*ms
ampa_nmda_ratio = 0.3 
tau_mem = 20*ms
vr = -70*mV
vI = -80*mV
vE = 0*mV
v_thr_r = -50*mV
d_thr = 100*mV
tau_thr = 5*ms

wee = 0.1
wei = 0.1
wie = 1
wii = 1

sparseness = 0.1 

#Neuron model 
eqs = '''
dv/dt = ((vr-v)+gE*(vE-v)+g_gaba*(vI-v))/tau_mem : volt
gE = ampa_nmda_ratio * g_ampa + (1-ampa_nmda_ratio) * g_nmda : 1
dg_nmda/dt = (-g_nmda + g_ampa)/tau_nmda : 1
dg_ampa/dt = -g_ampa/tau_ampa : 1
dg_gaba/dt = -g_gaba/tau_gaba : 1
dv_thr/dt = (-v_thr + v_thr_r)/tau_thr : volt 
'''
reset = '''
v = vr
v_thr += d_thr
'''

#Neuron group
G = NeuronGroup(NE+NI, eqs, threshold='v>v_thr', reset=reset, method='euler')
SE = G[:NE]
SI = G[NE:]

#Plasticity model
#parameters
w_max = 20
w_min = 0
eta = 0.01

# Rule 2348 - f19bec85c2d4104a767b4b821891852d 
rule = [0.03947454, 0.08333279, 1.56211138, -1.8362025, 0.3778016, -1.10496151,
        0.07777745, 0.09157054, 0.60514206, -0.82029539, 1.11860669, -0.92477429,
        0.09520964, 0.04604627, -1.27739811, 1.18037236, 1.04156876, -1.39219487,
        0.02658731, 0.07739804, -1.15393496, 0.75379592, 0.75430721, 1.92275465]

tau_preSTDP_EE, tau_postSTDP_EE, alpha_EE, beta_EE, gamma_EE, kappa_EE, tau_preSTDP_EI, tau_postSTDP_EI, alpha_EI, beta_EI, gamma_EI, kappa_EI, tau_preSTDP_IE, tau_postSTDP_IE, alpha_IE, beta_IE, gamma_IE, kappa_IE, tau_preSTDP_II, tau_postSTDP_II, alpha_II, beta_II, gamma_II, kappa_II  = load_plasticity_rule(rule)

eqs_STDP_EE ='''
w : 1
dApre/dt = -Apre/tau_preSTDP_EE / second : 1 (event-driven)
dApost/dt = -Apost/tau_postSTDP_EE / second : 1 (event-driven)
'''

eqs_STDP_EI ='''
w : 1
dApre/dt = -Apre/tau_preSTDP_EI / second : 1 (event-driven)
dApost/dt = -Apost/tau_postSTDP_EI  / second : 1 (event-driven)
'''

eqs_STDP_IE ='''
w : 1
dApre/dt = -Apre/tau_preSTDP_IE / second : 1 (event-driven)
dApost/dt = -Apost/tau_postSTDP_IE  / second : 1 (event-driven)
'''

eqs_STDP_II ='''
w : 1
dApre/dt = -Apre/tau_preSTDP_II / second : 1 (event-driven)
dApost/dt = -Apost/tau_postSTDP_II / second : 1 (event-driven)
'''

#Synapses with plasticity

synEE = Synapses(SE, SE, model=eqs_STDP_EE,
                    on_pre='''
                    Apre += 1
                    w = clip(w + eta * (alpha_EE + kappa_EE * Apost), w_min, w_max)
                    g_ampa += w
                    ''',
                    on_post='''
                    Apost += 1
                    w = clip(w + eta * (beta_EE + gamma_EE * Apre), w_min, w_max)
                    ''' )
synEE.connect(p=sparseness)
synEE.w = wee

synEI = Synapses(SE, SI, model=eqs_STDP_EI,
                    on_pre='''
                    Apre += 1
                    w = clip(w + eta * (alpha_EI + kappa_EI * Apost), w_min, w_max)
                    g_ampa += w
                    ''',
                    on_post='''
                    Apost += 1
                    w = clip(w + eta * (beta_EI + gamma_EI * Apre), w_min, w_max)
                    ''' )
synEI.connect(p=sparseness)
synEI.w = wei

synIE = Synapses(SI, SE, model=eqs_STDP_IE,
                    on_pre='''
                    Apre += 1
                    w = clip(w + eta * (alpha_IE + kappa_IE * Apost), w_min, w_max)
                    g_gaba += w
                    ''',
                    on_post='''
                    Apost += 1
                    w = clip(w + eta * (beta_IE + gamma_IE * Apre), w_min, w_max)
                    ''' )
synIE.connect(p=sparseness)
synIE.w = wie

synII = Synapses(SI, SI, model=eqs_STDP_II,
                    on_pre='''
                    Apre += 1
                    w = clip(w + eta * (alpha_II + kappa_II * Apost), w_min, w_max)
                    g_gaba += w
                    ''',
                    on_post='''
                    Apost += 1
                    w = clip(w + eta * (beta_II + gamma_II * Apre), w_min, w_max)
                    ''' )
synII.connect(p=sparseness)
synII.w = wii

# ### FOR STATIC NETWORK ###
# #Synapses
# synEE = Synapses(SE, SE, 'w : 1', on_pre='g_ampa += w')
# synEE.connect(p=sparseness)
# synEE.w = wee

# synEI = Synapses(SE, SI, 'w : 1', on_pre='g_ampa += w')
# synEI.connect(p=sparseness)
# synEI.w = wei

# synIE = Synapses(SI, SE, 'w : 1', on_pre='g_gaba += w')
# synIE.connect(p=sparseness)
# synIE.w = wie

# synII = Synapses(SI, SI, 'w : 1', on_pre='g_gaba += w')
# synII.connect(p=sparseness)
# synII.w = wii

N_input = 10000
radius = 8
background_rate = 10
weight_poisson = 0.075

# bg_stim = TimedArray(np.ones(int(duration*1000), dtype=int)*background_rate*Hz, dt=1*ms)
P_ext_bg = PoissonGroup(N_input, rates=background_rate*Hz) #'bg_stim(t)'

sources, targets = PtE.nonzero()
synPE = Synapses(P_ext_bg, SE, 'w : 1', on_pre='g_ampa += w')
synPE.connect(i=sources, j=targets)
synPE.w = weight_poisson

sourcesI, targetsI = PtI.nonzero()
synPI = Synapses(P_ext_bg, SI, 'w : 1', on_pre='g_ampa += w')
synPI.connect(i=sourcesI, j=targetsI)
synPI.w = weight_poisson

# sensory information for the network - stimulation based on the ball position
grid_size = 64

spike_monitor = SpikeMonitor(SE)

df = pd.read_csv("ball_position_seed2_1000trials.csv")
coordinate_list = df.values[:,1:3]
print(len(coordinate_list))
print(coordinate_list)

duration = len(coordinate_list) * 200*ms
stimulus_length = int(duration / 10 / ms)
print(f'Duration:{duration}')
print(f'Stim length:{stimulus_length}')

indices = np.ones(stimulus_length * 81) 
print(indices.shape)  # Should be (100 * 81,)

for i, (x, y) in enumerate(coordinate_list):
    _, index_values = get_surrounding_indices(x, y)

    if len(index_values) < 81:
        index_values += [-1] * (81 - len(index_values))
    
    for repeat in range(20):
        start_idx = (i * 20 + repeat) * 81 
        end_idx = start_idx + 81
        indices[start_idx:end_idx] = index_values

print(indices.shape)  # Should be (100 * 81,)

print(indices)

stimulus_spike_times = np.zeros(stimulus_length * 81)

epsilon = 1e-5
for i in range(stimulus_length): 
    for z in range(81):
        idx = (i * 81) + z  # Correct index calculation
        stimulus_spike_times[idx] = i * 10 + epsilon*np.random.uniform(0, 10)# Generate spikes every 10 ms


print(stimulus_spike_times)

indices = np.array(indices)
stimulus_spike_times = np.array(stimulus_spike_times)
stimulus_spike_times += 60000

valid_mask = indices != -1

indices = indices[valid_mask]
stimulus_spike_times = stimulus_spike_times[valid_mask]

indices = indices.tolist()
stimulus_spike_times = stimulus_spike_times.tolist()

spike_gen = SpikeGeneratorGroup(NE, times=stimulus_spike_times*ms, indices=indices)

syn_stim = Synapses(spike_gen, SE, 'w : 1', on_pre='g_ampa += w')  
syn_stim.connect(j='i')
syn_stim.w = 1.5

duration = ((len(coordinate_list) * 200) + 60000)*ms

run(duration, report='text')

In [ ]:
NE = 64 * 64  
grid_size = 64
bin_size = 0.2  # 200 ms 
duration = (len(coordinate_list) * 0.2) 
num_bins = int(duration / bin_size)

# Process spike times in chunks
spike_trains = spike_monitor.spike_trains()
bin_edges = np.linspace(0, duration, num_bins + 1)
firing_rates = np.zeros((NE, num_bins), dtype=float)

for i in range(NE):
    spike_times = spike_trains.get(i, [])
    if len(spike_times) > 0:

        hist, _ = np.histogram(spike_times, bins=bin_edges)
        firing_rates[i] = hist / bin_size  # Convert to Hz 

    del spike_times

# Normalize firing rates using the global maximum firing rate
global_max = firing_rates.max()
if global_max > 0:
    firing_rates /= global_max

firing_rate_grids = firing_rates.T.reshape(num_bins, grid_size, grid_size)

print(firing_rates.shape)
print(firing_rate_grids.shape)


(4096, 9558)
(9558, 64, 64)


In [ ]:
df = pd.DataFrame(np.array(firing_rates), 
                  columns=[f'Bin_{b}' for b in range(num_bins)])

# save
# df.to_csv("firing_rates_normalized_f19bec85_2000trials.csv", index=False)